In [1]:
#Spam filteting
import nltk
print(nltk.__version__)

3.5


In [60]:
import numpy as np
import pandas as pd
import sys
import sklearn
print("numpy version: {}".format(np.__version__))
print("pandas version: {}".format(pd.__version__))
print("sklearn version: {}".format(sklearn.__version__))
print("sys version: {}".format(sys.version))


numpy version: 1.19.0
pandas version: 1.0.5
sklearn version: 0.23.1
sys version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


## 1.Load the dataset

In [61]:
#dounble # highlights the markdown cell. for the above cell
#nowe lets read the file 
df = pd.read_table('SMSSpamCollection',header= None, encoding = 'utf-8')
# If we do not pass headee r = None, pandas will read from the second line.
print(type(df))

df.info()
print('______')
print(df.head())

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
______
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [62]:
A=np.array([1,1,2,3,2,3,4,])
df9=pd.DataFrame(A)
df9[0].value_counts() # .value_counts() works on series and not on the dataframe

3    2
2    2
1    2
4    1
Name: 0, dtype: int64

In [63]:
#Lets check out the count of ham and spam

classes = df[0] # I tried to use class, came to know that it is a reserved key word

classes.value_counts()

ham     4825
spam     747
Name: 0, dtype: int64

# it is a skewed distribution
### 2. Pre-processing the data

In [64]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

Y = encoder.fit_transform(classes)
print(Y[:10])
# NOw ew have converted the classed intp binaries

[0 0 1 0 0 1 0 0 1 1]


In [65]:
text_msg = df[1]
text_msg[0:10]

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object

## Some transformation of the data such as 
# i) we want to replace any email as 'email'
# ii) any url with 'url' 
# iii) phone numbers
# iv ) currency symbols

In [66]:


# use regular expressions to replace email addresses, URLs, phone numbers, other numbers

# Replace email addresses with 'email'
processed = text_msg.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

print(processed[8])


WINNER!! As a valued network customer you have been selected to receivea moneysymbnumbr prize reward! To claim call numbr. Claim code KLnumbr. Valid numbr hours only.


In [67]:


# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')



In [68]:
processed[2]

'Free entry in numbr a wkly comp to win FA Cup final tkts numbrst May numbr Text FA to numbr to receive entry question std txt rate T C s apply numbrovernumbr s'

## Change all the corpus to lower case

In [69]:
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [70]:
# Remove the stopwords
from nltk.corpus import stopwords
# print(stopwords)
stopwords = set(stopwords.words('english'))

print(stopwords, end= ' ')
print(type(stopwords))

{'between', 'ours', "wouldn't", 'here', "you're", 'won', 'when', 'myself', 'through', 'were', 'theirs', 'doing', "mightn't", "needn't", 'this', 'very', 'by', 'then', 'themselves', 'whom', 'you', 'himself', 'own', 'or', 'i', 'can', 'nor', 'should', 'out', "wasn't", 'weren', 'mustn', 'his', "hadn't", "didn't", "hasn't", 'don', 'have', 'he', 'hadn', 'our', 'her', 'those', 'wouldn', 'such', 'my', 'was', 'before', 'only', 'm', 'they', 'is', "aren't", "that'll", "don't", 'a', 'with', "you'll", 'again', 'why', 'each', 'd', "you've", "shouldn't", "should've", 'will', 'if', "shan't", 'few', 'its', 'some', 'most', 'over', 'other', 'which', 'had', 'but', 'further', 'does', 'in', "she's", 'while', 'there', 've', 'she', 'an', 'ma', 'both', "it's", 'any', 'above', 'and', 'during', 'it', 'off', 'all', 'did', 's', 'o', 'these', 'more', 'who', 'hasn', 'yourselves', 'their', 'on', 'them', 'him', 'yours', 'hers', 'shan', 'ain', 'once', 'shouldn', 'how', "doesn't", 'until', "isn't", 'having', 'about', 'ju

In [71]:
from nltk.tokenize import sent_tokenize,word_tokenize

In [72]:
def filter(sent):
    filtered = dict()
    for i in processed:
        words = word_tokenize(i)

        for word in words:
            rmv=[]
            if word not in stopwords:
                rmv.append(word)
                filtered[i]= rmv
                rmv=[]
    return filtered
    

In [73]:
# filter(processed[1])

In [74]:
processed_rm_stopwords = list(map(lambda x: ' '.join(term for term in x.split() if term not in stopwords),processed))
processed_rm_s= processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stopwords))

## The above and the below cell demonstrates that we can apply the map function to remove the stopwords in two ways. or by using map function in two ways

In [75]:
print(processed_rm_stopwords[0:5])
print(processed_rm_s[0:5])
print(f"processed_rm_s: {type(processed_rm_s)}")
print(f"processed_rm_stopwords{type(processed_rm_stopwords)}")
df_new=pd.DataFrame(processed_rm_stopwords)
print(type(df_new))
print(df_new[0:5])


['go jurong point crazy available bugis n great world la e buffet cine got amore wat', 'ok lar joking wif u oni', 'free entry numbr wkly comp win fa cup final tkts numbrst may numbr text fa numbr receive entry question std txt rate c apply numbrovernumbr', 'u dun say early hor u c already say', 'nah think goes usf lives around though']
0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry numbr wkly comp win fa cup final tk...
3                  u dun say early hor u c already say
4               nah think goes usf lives around though
Name: 1, dtype: object
processed_rm_s: <class 'pandas.core.series.Series'>
processed_rm_stopwords<class 'list'>
<class 'pandas.core.frame.DataFrame'>
                                                   0
0  go jurong point crazy available bugis n great ...
1                            ok lar joking wif u oni
2  free entry numbr wkly comp win fa cup final tk...
3                u dun sa

# After removing the stopwords lets do stemming using a porter stemmer

In [76]:
ps = nltk.PorterStemmer()
stemmed = list(map(lambda x: " ".join(ps.stem(term) for term in x.split()), processed_rm_stopwords))
print(stemmed[0:5])
df_new_stemmed = pd.DataFrame(stemmed)
print(df_new_stemmed)

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat', 'ok lar joke wif u oni', 'free entri numbr wkli comp win fa cup final tkt numbrst may numbr text fa numbr receiv entri question std txt rate c appli numbrovernumbr', 'u dun say earli hor u c alreadi say', 'nah think goe usf live around though']
                                                      0
0     go jurong point crazi avail bugi n great world...
1                                 ok lar joke wif u oni
2     free entri numbr wkli comp win fa cup final tk...
3                   u dun say earli hor u c alreadi say
4                  nah think goe usf live around though
...                                                 ...
5567  numbrnd time tri numbr contact u u moneysymbnu...
5568                            ü b go esplanad fr home
5569                                  piti mood suggest
5570  guy bitch act like interest buy someth els nex...
5571                                     rofl true name

[

# Now we can move on with generating features by tokenizing each sentence or each message

In [77]:
# Creating a bag of words
all_words =[] # this will have all the words in our dataframe (processed)

for item in stemmed:
    
    words  = word_tokenize(item)
    for word in words:
        all_words.append(word)
    
all_words = nltk.FreqDist(all_words) # unique words


In [78]:
len(all_words)

6579

In [79]:
# Lets find out some common words
print("most common words {}".format(all_words.most_common(10)))

most common words [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303)]


In [80]:
# Our feature will have words from these most common words. Lets use 1500 most common words as features. Now for every word in every msg
# we will have to create a vector of dimension 1500 (rows) with 1 in the word present and o elsewhere.
all_words_common = all_words.most_common(1500)
# all_words.keys()
features = all_words_common[0:1500]
features_list=[ key for  key, val in features]
print(features_list, end = ' ')

['numbr', 'u', 'call', 'go', 'get', 'ur', 'gt', 'lt', 'come', 'moneysymbnumbr', 'ok', 'free', 'day', 'know', 'love', 'like', 'got', 'time', 'good', 'want', 'text', 'send', 'txt', 'need', 'one', 'today', 'take', 'ü', 'see', 'stop', 'home', 'think', 'repli', 'r', 'lor', 'sorri', 'still', 'tell', 'n', 'numbrp', 'back', 'mobil', 'da', 'dont', 'make', 'k', 'week', 'pleas', 'phone', 'say', 'hi', 'work', 'new', 'pl', 'later', 'hope', 'miss', 'ask', 'co', 'meet', 'msg', 'messag', 'night', 'dear', 'c', 'wait', 'happi', 'well', 'tri', 'give', 'great', 'much', 'thing', 'claim', 'oh', 'min', 'wat', 'hey', 'number', 'na', 'friend', 'thank', 'ye', 'way', 'www', 'let', 'e', 'prize', 'feel', 'even', 'right', 'tomorrow', 'wan', 'alreadi', 'pick', 'cash', 'said', 'care', 'b', 'amp', 'yeah', 'im', 'leav', 'realli', 'tone', 'babe', 'win', 'life', 'morn', 'find', 'last', 'sleep', 'servic', 'keep', 'sure', 'use', 'anyth', 'uk', 'buy', 'would', 'year', 'start', 'contact', 'lol', 'also', 'urgent', 'nokia', 'w

## NOW ! since we have defined the features, the next thing is to find whether the feature is there or not in our processed messages. We need to define a function to find whether each word in the message is thers in the feature list of not

In [118]:
# lets define feature finding function
def find_feature(message):
    
    words = word_tokenize(message)
    
    feature= dict()
    
    for word in features_list:
        
        feature[word]=word in words
    return feature
        
features = find_feature(stemmed[0])
# print(features)
# print(len(features))


for key, val in features.items():
    if val == True:
        print (key)
    

go
got
n
great
wat
e
world
point
avail
crazi
bugi
la
cine


In [119]:
print(stemmed[0])

go jurong point crazi avail bugi n great world la e buffet cine got amor wat


In [120]:
messages = list(zip(stemmed,Y))
# list(messages)

In [121]:
df_N = pd.DataFrame(messages)
df_N.head
print(type(df_N))

<class 'pandas.core.frame.DataFrame'>


In [122]:
[key, label for (key, label) in df_N[0:5]]

SyntaxError: invalid syntax (<ipython-input-122-a3ee52950953>, line 1)

In [123]:
#define a seed 

seed = 1
np.random.seed =seed
np.random.shuffle(messages)
# print(df_N[0:5])

In [124]:
featureset = [(find_feature(text),label) for (text, label) in messages]

In [125]:
print(type(featureset))
print(len(featureset))

<class 'list'>
5572


## Preparing train and test data

In [126]:
from sklearn.model_selection import train_test_split

In [127]:
training, testing = train_test_split(featureset, test_size=0.2, random_state = seed)

In [128]:
print(len(training))
print(type(training))

4457
<class 'list'>


In [129]:
print("length of training set: {}".format(len(training)))
print("length of testing set: {}".format(len(testing)))

length of training set: 4457
length of testing set: 1115


In [130]:
training[0]

({'numbr': False,
  'u': False,
  'call': False,
  'go': False,
  'get': False,
  'ur': False,
  'gt': False,
  'lt': False,
  'come': False,
  'moneysymbnumbr': False,
  'ok': False,
  'free': False,
  'day': False,
  'know': False,
  'love': False,
  'like': False,
  'got': False,
  'time': False,
  'good': False,
  'want': False,
  'text': False,
  'send': False,
  'txt': False,
  'need': False,
  'one': False,
  'today': False,
  'take': False,
  'ü': False,
  'see': False,
  'stop': False,
  'home': False,
  'think': False,
  'repli': False,
  'r': False,
  'lor': False,
  'sorri': False,
  'still': False,
  'tell': False,
  'n': False,
  'numbrp': False,
  'back': False,
  'mobil': False,
  'da': False,
  'dont': False,
  'make': True,
  'k': False,
  'week': False,
  'pleas': False,
  'phone': False,
  'say': False,
  'hi': False,
  'work': False,
  'new': False,
  'pl': False,
  'later': False,
  'hope': False,
  'miss': False,
  'ask': False,
  'co': False,
  'meet': False,
  

## Scikit learn classifiers


In [173]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

In [174]:
names = ["K nearest neighbors","decision tree","random forest","SVC","Logistics regression","SGD","MultinomialNB"]
classifiers = [KNeighborsClassifier(),
               DecisionTreeClassifier(),
               RandomForestClassifier(),
               SVC(kernel = 'linear'),
               LogisticRegression(),
              SGDClassifier(max_iter=100),
               MultinomialNB()]
models = zip(names,classifiers)
list(models)



[('K nearest neighbors', KNeighborsClassifier()),
 ('decision tree', DecisionTreeClassifier()),
 ('random forest', RandomForestClassifier()),
 ('SVC', SVC(kernel='linear')),
 ('Logistics regression', LogisticRegression()),
 ('SGD', SGDClassifier(max_iter=100)),
 ('MultinomialNB', MultinomialNB())]

In [149]:

for (name, model) in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [96]:
# training, testing

In [176]:
# Define models to train
from nltk.classify.scikitlearn import SklearnClassifier
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]
models = zip(names, classifiers)
from nltk.classify.scikitlearn import SklearnClassifier
for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 94.7085201793722
Decision Tree Accuracy: 96.32286995515696
Random Forest Accuracy: 99.01345291479821
Logistic Regression Accuracy: 98.65470852017937
SGD Classifier Accuracy: 98.56502242152466
Naive Bayes Accuracy: 98.65470852017937
SVM Linear Accuracy: 98.29596412556054


In [152]:
# #wrap models in NLTK
# from nltk.classify.scikitlearn import SklearnClassifier
# for name, model in models:
#     nltk_model = SklearnClassifier(model)
#     nltk_model.train(training)
#     accuracy = nltk.classify.accuracy(nltk_model, testing)*100
#     print(f"accuracy for {name} is : {accuracy}")

In [150]:
nl=SklearnClassifier(SGDClassifier(max_iter=100))
nl.train(training)

<SklearnClassifier(SGDClassifier(max_iter=100))>

In [151]:
accuracy_nl = nltk.classify.accuracy(nl, testing)*100
print("{} Accuracy: {}".format('SGD', accuracy_nl))

SGD Accuracy: 98.29596412556054


## Lets try ensemble method, as a spam clasifier has to have a higher accuracy and having an ensemble method m ay help us get higher accuracy. Lets try the voting  classifier 
Each of our classifier will have their individual vote for each email/message and the highest vote wins. We need to have an odd number of classifier for this.  Note, nltk will see the ensembled model as one  model and it will not see the underlying 7 models of ours.
# hard voting and soft voting. Hard voting is bonary an soft takes the individual probabiities for each class and go with the average value

In [206]:
models = zip(names, classifiers)
models


In [192]:
classifiers

[KNeighborsClassifier(),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 LogisticRegression(),
 SGDClassifier(max_iter=100),
 MultinomialNB(),
 SVC(kernel='linear')]

In [205]:
estimators= list(models)
print(estimators)

[]


In [207]:
from sklearn.ensemble import VotingClassifier
nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = list(models), voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
# accuracy = nltk.classifiy.accuracy(nltk_ensemble, test)*100
# print(f"the accuracy of the ensemble model is: {accuracy}")

<SklearnClassifier(VotingClassifier(estimators=[('K Nearest Neighbors', KNeighborsClassifier()),
                             ('Decision Tree', DecisionTreeClassifier()),
                             ('Random Forest', RandomForestClassifier()),
                             ('Logistic Regression', LogisticRegression()),
                             ('SGD Classifier', SGDClassifier(max_iter=100)),
                             ('Naive Bayes', MultinomialNB()),
                             ('SVM Linear', SVC(kernel='linear'))],
                 n_jobs=-1))>

In [166]:
nltk_ensemble

<SklearnClassifier(VotingClassifier(estimators=<zip object at 0x0000023BC9A01C48>, n_jobs=-1))>

In [211]:
accuracy = nltk.classify.accuracy(nltk_ensemble, testing)*100
print(f"the accuracy of the ensemble model is: {accuracy}")

the accuracy of the ensemble model is: 98.83408071748879


## the ensemble model has done almost similar to our best model which was RandomForest Usually, if none of our models are performing well, in that case ensemble model may come up with better and higher accuracy. Now, lets test some of the results

In [213]:
print(testing[0]) # as you can see testing is a tuple if feature_list and the label for the message

({'numbr': True, 'u': True, 'call': True, 'go': False, 'get': False, 'ur': True, 'gt': False, 'lt': False, 'come': False, 'moneysymbnumbr': False, 'ok': False, 'free': False, 'day': False, 'know': False, 'love': False, 'like': False, 'got': False, 'time': False, 'good': False, 'want': False, 'text': False, 'send': False, 'txt': False, 'need': False, 'one': False, 'today': False, 'take': False, 'ü': False, 'see': False, 'stop': False, 'home': False, 'think': True, 'repli': False, 'r': True, 'lor': False, 'sorri': False, 'still': False, 'tell': False, 'n': False, 'numbrp': False, 'back': False, 'mobil': False, 'da': False, 'dont': False, 'make': True, 'k': False, 'week': False, 'pleas': False, 'phone': False, 'say': False, 'hi': False, 'work': False, 'new': False, 'pl': False, 'later': False, 'hope': False, 'miss': False, 'ask': False, 'co': False, 'meet': False, 'msg': False, 'messag': False, 'night': False, 'dear': False, 'c': False, 'wait': False, 'happi': False, 'well': False, 'tri':

In [214]:
text_feature, label = zip(*testing)

In [215]:
predictions = nltk_ensemble.classify_many(text_feature)

In [217]:
print(classification_report(label, predictions))

# pd.DataFrame(confusion_matrix(label, predictions), index=[['actual','actual'],['ham','spam']],
#             columns = [['predicted','predicted'],['ham','spam']])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       949
           1       0.99      0.93      0.96       166

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [218]:
pd.DataFrame(confusion_matrix(label, predictions), index=[['actual','actual'],['ham','spam']],
            columns = [['predicted','predicted'],['ham','spam']])

predicted     
                  ham spam
actual ham        948    1
       spam        12  154

## SO, overall our model did a good job as desired. The important thing is to learn how to make NLTK and sklearn libraries work together. We can expect a better performance by using a larger feature list. Instead of using only 1500 words as in our case, we can use all the words, and hope to get a better performance